# Final Project: Wind Integration in ISO New England
**Eliza Cohn and Julia Simpson**

**EEEL 4220 Fall 2023**

<font color='blue'>**Key information in blue**</font>

# Directions

## Introduction
You take the position of a policy maker to provide policy suggestions to decarbonize ISO New England and are given the attached one-bus system model and. Details about this system model are explained in the attached paper. Note the model in this paper has 8 buses with no transmission limits, so it is equivalent to a single-bus model (ISO New England has minor congestion issues).
ISONE expects to install a total wind generation capacity of 12,000 MW by 2030 and would like to study how to best integrate wind generation. You are provided with one week (168 hours) of data including wind generation capacity factors and demand. The wind generator capacity factor is a normalized value between zero to one representation ratio between the actual wind generation and the installed wind capacity over each hour.

## Project Objective

Please finish a report summarizing how to best integrate up to 12,000 MW of wind generation into the system and how the wind generation would impact the system operating cost. You should explore three levels of wind capacity:
- Low: 2,000 MW
- Medium: 6,000 MW • High: 12,000 MW

You need to implement a unit commitment simulation and explore approaches that could improve the wind integration ratio, these approaches include but are not limited to
- Add carbon tax that increases the cost of traditional generators. You can use the EIA website1 to look up emission factors of different fuel types.
- Add tax incentives for wind integration, in this case, you should add a negative cost of wind generation wt to the objective function.
- Retirement of coal power plants.

At each scenario, use the following indexes to compare the system performance
- The total generation cost.
- The wind curtailment ratio - the ratio between the wind that was not integrated into the system
(Wt − wt) to the total wind capacity available (Wt).
- Profits received by generators.
- The average electricity price.
- The profit received by generators.

Finally, your project and report should conclude your recommendation of the best solution that balances electricity cost, system-wide carbon emission, and generator revenue. Note that there will not be a unique solution, and you should use your own judgements about which one you think is the best.

## Directions
1. The major effort in this project is to program a unit commitment model. **Wind generation should be modeled as a zero marginal cost resource (or negative marginal cost if it has tax credits) and can be curtailed**, i.e., wt ≤ WAt, where W are installed wind capacity, At is the time-variant capacity factor given in the data, and wt is the actual wind power integrated into the grid. w should be included in the nodal power balance as a generator. For the rest, please refer to the complete unit commitment formulation in the attachment.
2. You are given a full week’s profile but performing a unit commitment with 168 hours might be overwhelming for your computer. Instead, you should perform sequential daily unit commitments of 24 hours, and pass decision variables as the starting point (in which they become coefficients) for your next unit commitment. These will include gi,t and ui,t. You must also update SUi, and SDi accordingly based on the start-up/down status. For your first operating day, you can assume all generators have been off long enough and set these parameters.
3. The generator cost data includes a quadratic term and a linear term. **You can either use mixed- integer quadratic programming to solve it directly or apply piece-wise linear (five segments per generator max) to the cost curve and convert the problem into MILP.**
4. You can assume all generators are off at the start of each scenario with a zero must-down time, i.e., you can start any generators at the start of the horizon.
5. After implementing a functional unit commitment model, please adjust the installed wind power capacity and other intervention methods, observe the result from each scenario, and summarize the results in your report.

## Appendix - Unit Commitment Formulation

Indexes
- $t$: Time period index $t ∈ {1,2,...,T}$, a total of $T$ time periods. 
- $b$: Bus index $b ∈ {1,2,...,B}$, a total of $B$ buses.
- $i$: Generator index $i ∈ {1,2,...,I}$, a total of $I$ generators.
- $l$: Line index $l ∈ {1,2,...,L}$, a total of $L$ lines.

Cost parameters
- $QC_i$: the quadratic generation cost term for generator $i$.
- $LC_i$: the linear generation cost term for generator $i$.
- $NLC_i$: no load cost for generator $i$.
- $SUC_i$: start-up cost for generator $i$.

Unit constraints parameters
- $Gmax_i$: maximum generation of generator $i$
- $Gmin_i$: minimum generation of generator $i$
- $Tup_i$: minimum up time of generator $i$
- $Tdn_i$: minimum down time of generator $i$
- $RR_i$: ramp rate of generator $i$
- $SU_i$: number of time periods generator $i$ must stay up since the start of the optimization horizon.
- $SD_i$: number of time periods generator $i$ must stay down since the start of the optimization horizon.

Demand and renewable parameters
- $W$: installed wind capacity
- $α_t$: wind capacity factor during time period $t$
- $D_t$: system demand during time period $t$

Continuous decision variables
- $g_{i,t}$: production of generator $i$ during time period $t$
- $r_{i,t}$: hourly reserve generator $i$ could provide during time period $t$
- $w_t$: wind power generation during time period $t$
- $s_t$: slack variable representing cost of load shedding during time period $t$

Binary decision variables
- $u_{i,t}$: 1 if generator $i$ is on during time period $t$, otherwise zero
- $v_{i,t}$: 1 if generator $i$ is turned on at the start of period $t$, otherwise zero 
- $z_{i,t}$: 1 if generator $i$ is turned off at the start of period $t$, otherwise zero

The objective function minimizes total generation costs. Note that the last term is about the slack variable $s_t$ which represents the cost of load shedding which is priced at $9,000\$/MWh$.

$$ min \sum\limits_{t} \sum\limits_{i} (QC_ig_{i,t}^2 +LC_ig_{i,t} + NLC_iu_{i,t} +SUC_iv_{i,t}) + (9000)s_t $$

Generation limit constraint $ (i ∈ {1,2,...,I}, t ∈ {1,2,...,T}) $ 
$$ Gmin_iu_{i,t} \leq g_{i,t} \leq Gmax_iu_{i,t}$$


Generation ramp constraint, the minimum generation limit adds to the ramp-up limit when the generator is turned on or off $(i ∈ {1,2,...,I}, t ∈ {1,2,...,T}) $

$$ −Gmin_iz_{i,t} − RR_i ≤ g_{i,t} − g_{i,t}−1 ≤ RR_i + Gmin_iv_{i,t} $$

Generator start-up and shut-down logic $(i ∈ {1,2,...,I}, t ∈ {1,2,...,T})$
$$ v_{i,t} − z_{i,t} = u_{i,t} − u_{i,t−1} $$
$$ v_{i,t} + z_{i,t} ≤ 1 $$

Generator minimum up time constraint, note the use of the alternative time index τ , and the constraint only accounts for periods beyond the required must on or off time since the start of the optimization horizon $(i ∈ {1,2,...,I})$
<font color='red'>$$ \sum\limits_{τ=max{t−Tup_i+1,1}}^t v_{i,τ} ≤ u_{i,t},t ∈ {SU_i,...,T}$$ 
<font color='red'>$$ \sum\limits_{τ=max{t−Tdn_i+1,1}}^t z_{i,τ} ≤ 1-u_{i,t},t ∈ {SD_i,...,T}$$ 

accompanied by the must-stay on or off constraints, which passes the on/off requirement from the
previous day to the current day
$$ \sum\limits_{t=1}^{SU_i} u_{i,t} = SU_i $$
$$ \sum\limits_{t=1}^{SD_i} u_{i,t} = 0 $$


Power balance, the dual variable λt associated with this constraint is the system price
$$ \sum\limits_{i} g_{i,t} + w_t + s_t = D_t : \lambda_t $$ 


System reserve (3+5 rule: hourly reserve amount must equal to 3% of demand and 5% of integrated wind)
$$ \sum\limits_{i} r_{i,t} \geq (3\%) D_t + (5\%)w_t $$
$$ r_{i,t} \leq  Gmax_iu_{i,t} − g_{i,t} $$
$$ r_{i,t} \leq RR_i $$

Wind generation limit
$$ w_t \leq \alpha_tW $$

# Code for Project

## Data Import

In [1]:
import cvxpy as cp
import numpy as np
import gurobipy
import pandas as pd

gen = pd.read_csv('generator_data.csv')
load = pd.read_csv('wind_and_demand.csv')

## Parameter Definition

In [2]:
I = gen.shape[0]; # number of generators
T = 24; # number of hours

# unit parameters are 1 by 3 vectors for the three generators
QC      = gen['Dispatch Cost Quadratic\nCoefficient b \n($/MW^2h)'].ravel() # quadratic gen cost 
LC      = gen['Dispatch Cost Linear\nCoefficient a \n($/MWh)'].ravel()  # marginal generator cost
NLC     = gen['NoLoad\nCost ($)'].ravel()  # no load cost
SUC     = gen['StartUp \nCost ($)'].ravel() # start up cost
GMin    = gen['Minimum generation (MW)'].ravel()  # minium generation
GMax    = gen['Capacity \n(MW)'].ravel()  # maximum generation
RR      = gen['Ramp Rate\n(MW/hr)'].ravel()
Tup     = gen['MinUp\nTime (hr)'].ravel()
Tdn     = gen['MinDown \nTime (hr)'].ravel()

u0      = np.zeros(I)  # initial commitment status (all turned off)
LOAD    = load.Demand[0:24]

## Variable Definition

In [3]:
# the first index is generator, the second is period
g = cp.Variable((I,T), nonneg = True)   # generator dispatch
r = cp.Variable((I,T), nonneg = True)   # hourly reserve generator
w = cp.Variable((T), nonneg = True)   # wind power dispatch
s = cp.Variable((T), nonneg = True)   # slack variable for load shedding

u = cp.Variable((I,T), boolean = True)   # commitment status
v = cp.Variable((I,T), boolean = True)   # start-up status
z = cp.Variable((I,T), boolean = True)   # inverse of start-up status

## Defining Objective Function & Constraints

### Objective Function

$$ min \sum\limits_{t} \sum\limits_{i} (QC_ig_{i,t}^2 +LC_ig_{i,t} + NLC_iu_{i,t} +SUC_iv_{i,t}) + (9000)s_t $$

In [4]:
#obj = cp.Minimize(sum(sum(QC @ g**2) + sum(LC @ g) + sum(NLC @ u) + sum(SUC @ v) + sum(9000*s)))

In [5]:
obj = cp.Minimize(sum(sum(QC @ g**2 + LC @ g + NLC @ u + SUC @ v) + 9000*s)) 

### Constraints

In [6]:
# Initialize an empty constraint set
con_set = []  

Generation limit constraint $ (i ∈ {1,2,...,I}, t ∈ {1,2,...,T}) $ 
$$ Gmin_iu_{i,t} \leq g_{i,t} \leq Gmax_iu_{i,t}$$

In [10]:
for t in range(T): # go through each period
    for i in range(I): # go through each generator
        con_set.append(g[i][t] <= GMax[i] * u[i][t])  # maximum generation limits
        con_set.append(g[i][t] >= GMin[i] * u[i][t])  # minimum generation limits

Generation ramp constraint, the minimum generation limit adds to the ramp-up limit when the generator is turned on or off $(i ∈ {1,2,...,I}, t ∈ {1,2,...,T}) $

$$ −Gmin_iz_{i,t} − RR_i ≤ g_{i,t} − g_{i,t−1} ≤ RR_i + Gmin_iv_{i,t} $$

Generator start-up and shut-down logic $(i ∈ {1,2,...,I}, t ∈ {1,2,...,T})$
$$ v_{i,t} − z_{i,t} = u_{i,t} − u_{i,t−1} $$
$$ v_{i,t} + z_{i,t} ≤ 1 $$

In [41]:
## Eliza update: 
    # combined generator ramp constraints with start up shut down logic since within the same for loop
    # encoded t=0 logic following HW6 example

for t in range(T): # go through each period
    for i in range(I): # go through each generator
        con_set.append(g[i][t] - g[i][t-1] <= RR[i] + GMin[i] * v[i][t])  # minimum gen limit added to ramp limit when generator turned on or off
        con_set.append(g[i][t] - g[i][t-1] >= -GMin[i] * z[i][t] - RR[i])  # minimum gen limit added to ramp limit when generator turned on or off
        
        # Generator startup-shutdown logic
        if t==0:
            # for the first period, check with initial commitment status
            con_set.append(v[i][0] - z[i][0] == u[i][0] - u0[i])
            con_set.append(v[i][0] + z[i][0] <= 1)
        else:
            # for other periods, check difference between two commitment status
            con_set.append(v[i][t] - z[i][t] == u[i][t] - u[i][t-1])
            con_set.append(v[i][t] + z[i][t] <= 1)
        

<font color='red orange'>LEFT OFF HERE

Generator minimum up time constraint, note the use of the alternative time index τ , and the constraint only accounts for periods beyond the required must on or off time since the start of the optimization horizon $(i ∈ {1,2,...,I})$
<font color='red'>$$ \sum\limits_{\tau=max{t−Tup_i+1,1}}^t v_{i,τ} ≤ u_{i,t},t ∈ {SU_i,...,T}$$ 
<font color='red'>$$ \sum\limits_{\tau=max{t−Tdn_i+1,1}}^t z_{i,τ} ≤ 1-u_{i,t},t ∈ {SD_i,...,T}$$ 

accompanied by the must-stay on or off constraints, which passes the on/off requirement from the
previous day to the current day
$$ \sum\limits_{t=1}^{SU_i} u_{i,t} = SU_i $$
$$ \sum\limits_{t=1}^{SD_i} u_{i,t} = 0 $$

Power balance, the dual variable $\lambda_t$ associated with this constraint is the system price
$$ \sum\limits_{i} g_{i,t} + w_t + s_t = D_t : \lambda_t $$ 

In [44]:
con_set.append( LOAD == sum(g)+ w + s) # demand balance constraint accounting for wind generation and slack

System reserve (3+5 rule: hourly reserve amount must equal to 3% of demand and 5% of integrated wind)
$$ \sum\limits_{i} r_{i,t} \geq (3\%) D_t + (5\%)w_t $$
$$ r_{i,t} \leq  Gmax_iu_{i,t} − g_{i,t} $$
$$ r_{i,t} \leq RR_i $$

In [ ]:
con_set.append( sum(r[t]) >= 0.03*D[t] + 0.05*w[t]) #not sure if r should be r[i][t] or r[i] instead, but THINK sum all i over constant t
con_set.append( r[i][t] <= GMax[i]*u[i][t] - g[i][t]) 
con_set.append( r[i][t] <= RR[i]) 

Wind generation limit
$$ w_t \leq \alpha_tW $$

In [ ]:
con_set.append( w[t] <= alpha[t]*W ) #alpha is wind capacity factor, do we have that loaded in from data? W is installed wind capacity

## Solve and Check Results

In [ ]:
# Solve the problem
prob1 = cp.Problem(obj, con_set_1)
prob1.solve(solver = "GUROBI")
prob1.solve();

# Example Code from HW5

<font color='blue'>**Start of Problem 1 Part 1**</font>

Solution using Lagrangian multipliers from Lecture 4 Slide 89:
$$\ell(x_1,...,x_n,\lambda_1,...,\lambda_m,\mu_1,...,\mu_p) = f(x_1,...,x_n) + \Sigma_{i=1}^m \lambda_i*\omega(x_1,..,x_n) + \Sigma_{j=1}^p \mu_j*g_j(x_1,..,x_n)$$

For this case:

$$\ell(x,y,z,\lambda_1,...,\lambda_m,\mu_1,...,\mu_p) = f(x,y,z) + \Sigma_{i=1}^m \lambda_i*\omega(x,y,z) + \Sigma_{j=1}^p \mu_j*g_j(x,y,z)$$

Function to be minimized:

$$ f(x,y,z) = x^2 + 2x + 2y^2 + 5y + z $$

Equality constraint $\omega$: 

$$\omega = 5 - x - y - z $$

Also given: 

$$ 2x + z \geq 0 $$

Rearranging for inequality constraint $g$:
$$  -2x - z \leq 0 $$

Just have the 1 equality constraint $\omega$, thus only 1 $\lambda$ or $m$=1:

$$\ell(x,y,z,\lambda,\mu_1,...,\mu_p) = f(x,y,z) + \lambda*\omega(x,y,z) + \Sigma_{j=1}^p \mu_j*g_j(x,y,z)$$

Just have the 1 inequality constraint $g$, thus only 1 $\mu$ or $p$=1:

$$\ell(x,y,z,\lambda,\mu) = f(x,y,z) + \lambda*\omega(x,y,z) + \mu*g(x,y,z)$$

<font color='blue'>Thus Lagrangian is:

<font color='blue'>$$\ell(x,y,z,\lambda,\mu) = x^2 + 2x + 2y^2 + 5y + z + \lambda*[5 - x - y - z] + \mu*[-2x - z]$$

<font color='blue'>**Answer to Problem 1 Part 1**</font>

<font color='blue'>$$\ell(x,y,z,\lambda,\mu) = x^2 + 2x + 2y^2 + 5y + z + \lambda*[5 - x - y - z] + \mu*[-2x - z]$$

<font color='blue'>**Start of Problem 1 Part 2**</font>

from slide 88 of Lecture 4:

$$\ell(x,\lambda,\mu) = f(x) + \Sigma_{i=1}^m \lambda_i*\omega_i(x) + \Sigma_{j=1}^p \mu_j*g_j(x)$$

Karush Kuhn Tucker (KKT) condition (also listed on same slide):

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial x_i} \equiv \frac{\partial f(x)}{\partial x_i} +  \Sigma_{i=1}^m \lambda_i *\frac{\partial \omega_i(x)}{\partial x_i} + \Sigma_{j=1}^p \mu_j* \frac{\partial g_j(x)}{\partial x_i} = 0$$


$$\frac{\partial\ell(x,\lambda,\mu)}{\partial \lambda_k} \equiv \omega_k(x) = 0$$

$$ g_j(x) \leq 0$$

With complementary slackness conditions:

$$ \mu_j g_j(x) = 0 $$
$$ \mu_j \geq 0 $$

Thus KKT condition is as follows:

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial x_i} \equiv \frac{\partial [x^2 + 2x + 2y^2 + 5y + z]}{\partial x_i} +  \lambda *\frac{\partial [5 - x - y - z]}{\partial x_i} + \mu* \frac{\partial [ -2x - z ]}{\partial x_i} = 0$$

<font color='blue'>$$\frac{\partial\ell(x,\lambda,\mu)}{\partial \lambda} \equiv 5 - x - y - z = 0$$

<font color='blue'>$$ g_j(x)\equiv -2x - z \leq 0$$

With respect to x:

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial x} \equiv \frac{\partial [x^2 + 2x + 2y^2 + 5y + z]}{\partial x} +  \lambda *\frac{\partial [5 - x - y - z]}{\partial x} + \mu* \frac{\partial [ -2x - z ]}{\partial x} = 0$$

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial x} \equiv [2x + 2] +  \lambda *[-1] + \mu*[ -2] = 0$$

<font color='blue'>$$\frac{\partial\ell(x,\lambda,\mu)}{\partial x} \equiv 2x + 2 - \lambda - 2 \mu = 0$$

With respect to y:

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial y} \equiv \frac{\partial [x^2 + 2x + 2y^2 + 5y + z]}{\partial y} +  \lambda *\frac{\partial [5 - x - y - z]}{\partial y} + \mu* \frac{\partial [ -2x - z ]}{\partial y} = 0$$

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial y} \equiv [2*2y + 5] +  \lambda *[-1] + \mu* [0] = 0$$

<font color='blue'>$$\frac{\partial\ell(x,\lambda,\mu)}{\partial y} \equiv 4y + 5 - \lambda = 0$$

With respect to z:

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial z} \equiv \frac{\partial [x^2 + 2x + 2y^2 + 5y + z]}{\partial z} +  \lambda *\frac{\partial [5 - x - y - z]}{\partial z} + \mu* \frac{\partial [ -2x - z ]}{\partial z} = 0$$

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial z} \equiv [1] +  \lambda * [-1]  + \mu* [-1] = 0$$

<font color='blue'>$$\frac{\partial\ell(x,\lambda,\mu)}{\partial z} \equiv 1 -  \lambda  - \mu = 0$$

Plugging into complementary slackness conditions:

$$ \mu g(x) = \mu[-2x - z] = 0 $$

<font color='blue'>$$ \mu g(x) = -2x\mu - z\mu = 0 $$

<font color='blue'>$$ \mu \geq 0 $$

<font color='blue'>**Answer to Problem 1 Part 2**</font>

<font color='blue'>**In summary, KKT condition is as follows:**

<font color='blue'>$$\frac{\partial\ell(x,\lambda,\mu)}{\partial x} \equiv 2x + 2 - \lambda - 2 \mu = 0$$

<font color='blue'>$$\frac{\partial\ell(x,\lambda,\mu)}{\partial y} \equiv 4y + 5 - \lambda = 0$$

<font color='blue'>$$\frac{\partial\ell(x,\lambda,\mu)}{\partial z} \equiv 1 -  \lambda  - \mu = 0$$

<font color='blue'>$$\frac{\partial\ell(x,\lambda,\mu)}{\partial \lambda} \equiv 5 - x - y - z = 0$$

<font color='blue'>$$ g(x)\equiv -2x - z \leq 0$$

<font color='blue'>**With complementary slackness conditions:**

<font color='blue'>$$ \mu g(x) = -2x\mu - z\mu = 0 $$

<font color='blue'>$$ \mu \geq 0 $$

<font color='blue'>**Start of Problem 1 Part 3**</font>

First assume inequality constraint not binding (slide 94):
$$ \mu = 0 $$

Plugging into eqn wrt x:

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial x} \equiv 2x + 2 - \lambda - 2 \mu = 0$$

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial x} \equiv 2x + 2 - \lambda - 2 [0] = 0$$

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial x} \equiv 2x + 2 - \lambda = 0$$

Plugging into eqn wrt y:
$$\frac{\partial\ell(x,\lambda,\mu)}{\partial y} \equiv 4y + 5 - \lambda = 0$$

Plugging into eqn wrt z:
$$\frac{\partial\ell(x,\lambda,\mu)}{\partial z} \equiv 1 -  \lambda  - \mu = 0$$

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial z} \equiv 1 -  \lambda  - [0] = 0$$

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial z} \equiv 1 -  \lambda = 0$$

Solving eqn wrt z:

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial z} \equiv 1 -  \lambda = 0$$

$$ 1 -  \lambda = 0$$

$$ \lambda = 1$$

Solving eqn wrt x:

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial x} \equiv 2x + 2 - \lambda = 0$$

$$ 2x + 2 - [1] = 0$$

$$ 2x + 1 = 0$$

<font color='blue'>$$ x = -0.5$$

Solving eqn wrt y:

$$\frac{\partial\ell(x,\lambda,\mu)}{\partial y} \equiv 4y + 5 - \lambda = 0$$

$$ 4y + 5 - [1] = 0$$

$$ 4y + 4 = 0$$

<font color='blue'>$$ y = -1$$

Find z:

$$x + y + z = 5$$

$$z = 5 - x - y $$

$$z = 5 - (-0.5) - (-1)$$

<font color='blue'>$$z = 6.5$$

Check feasibility:
$$  -2x - z \leq 0 ?$$

$$  -2[-0.5] - [6.5] \leq 0 ?$$

$$  1 - [6.5] \leq 0 ?$$

$$  -5.5 \leq 0 $$

Thus, solution is feasible

Find value of objective function for $x = -0.5$, $y= -1$, $z=6.5$:

$$x^2 + 2x + 2y^2 + 5y + z$$

$$(-0.5)^2 + 2(-0.5) + 2(-1)^2 + 5(-1) + (6.5)$$

$$0.25 -1 + 2 -5 + 6.5$$

$$0.25 + 1 + 1.5$$

<font color='blue'>$$ 2.75$$

<font color='blue'>**Answer to Problem 1 Part 3**</font>

<font color='blue'>$x = -0.5$, $y= -1$, $z=6.5$ for objective function value of $2.75$

In [120]:
# check with CVXPY:
import cvxpy as cp

In [121]:
# Define variables
x = cp.Variable(nonneg = False)
y = cp.Variable(nonneg = False)
z = cp.Variable(nonneg = False)

In [122]:
# Create objective function
obj = cp.Minimize(x**2 + 2*x + 2*y**2 + 5*y + z)

In [123]:
con = [];
# Define inequality constraints 
con += [-2*x - z <= 0];
# Define equality constraints 
con += [5 - x - y -z == 0]

In [124]:
# Solve the problem
prob = cp.Problem(obj, con)

In [125]:
# call solver to solve the problem
prob.solve(verbose = True) # You may specify the solver here, 
                           # otherwise ECOS is used for LPs by default

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Oct 20 06:53:46 PM: Your problem has 3 variables, 2 constraints, and 0 parameters.
(CVXPY) Oct 20 06:53:46 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 20 06:53:46 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 20 06:53:46 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 20 06:53:46 PM: Compiling problem (target solver=OSQP).
(CVXPY) Oct 20 06:53:46 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing 

2.7500000000000004

In [126]:
prob.status

'optimal'

In [127]:
#print optimal value and variables
print('\033[94mOptimal value: ', f'{prob.value:.8}' +'\033[0m')
print('\033[94mOptimal variables: ', f'x={x.value:.5}, y={y.value:.5}, z={z.value:.5}'+'\033[0m')

Optimal value:  2.75
Optimal variables:  x=-0.5, y=-1.0, z=6.5


### Problem 2 Economic dispatch using CVXPY (20)

A utility company has three generators with the following cost functions (unit in \\$/h):
$$
\begin{align}
C_1(x_1) &= 0.01 x_1^2 + 20 x_1 + 100 \\
C_2(x_2) &= 0.02 x_2^2 + 25 x_2 + 200 \\
C_3(x_3) &= 0.03 x_3^2 + 30 x_3 + 300 
\end{align}
$$

And the minimum and maximum generator capacity are listed below:
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky"> </th>
    <th class="tg-0pky">Gen Min (MW) </th>
    <th class="tg-0pky">Gem Max (MW)</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">Generator 1</td>
    <td class="tg-0pky">100</td>
    <td class="tg-0pky">500</td>
  </tr>
  <tr>
    <td class="tg-0pky">Generator 2</td>
    <td class="tg-0pky">50</td>
    <td class="tg-0pky">300</td>
  </tr>
  <tr>
    <td class="tg-0pky">Generator 3</td>
    <td class="tg-0pky">0</td>
    <td class="tg-0pky">100</td>
  </tr>
</tbody>
</table>


We now practice to solve the optimization using CVXPY. Before we start, load the CVXPY package

In [83]:
import cvxpy as cp

First, we need to define variables:

In [84]:
# Define x1 as a variable
x1 = cp.Variable(nonneg = True)

It defines `x1` as a variable under the CVXPY class - `cp.Variable`. Since most of the time we will be dealing with nonnegative variables, it is convenient to simply enforce variables to be positive only with `(nonneg = True)`. 

1. Now define variables for generator 2 and 3 (5)

In [85]:
# Define x2 as a variable
x2 = cp.Variable(nonneg = True)
# Define x3 as a variable
x3 = cp.Variable(nonneg = True)

Next, define the objective function as a minimization problem, note that the first generator is already given.

2. Please fill in the rest two generators into the objective function (5)

In [95]:
# Fill in cost function for generator two and three
obj = cp.Minimize(
    0.01*cp.square(x1) + 20 * x1 + 100 + 
    0.02*cp.square(x2) + 25*x2 + 200 +
    0.03*cp.square(x3) + 30*x3 + 300 
                 )

Now define inequality constraints

In [108]:
con = [];
con += [x1 >= 100];
con += [x1 <= 500];

3. Define inequality constraints for generator 2 and 3 (5)

In [109]:
# Define constraints for generator 2
con += [x2 >= 50];
con += [x2 <= 300];
# Define constraints for generator 3
con += [x3 >= 0];
con += [x3 <= 100];

Finally, add the last constraint which enforces the sum of generatoin equals to the demand which is 500 MW. Note that all equality constraints must be defined using `==` instead of `=`.

4. Define the demand balance constraint (5)

In [110]:
# Define the demand balance constraint
con += [500 - x1 - x2 - x3 == 0]

So far we have defined the objective `obj` and the constraint set `con`. Now we assemble them together into a **Problem** with name `prob1`

In [111]:
# Solve the problem
prob1 = cp.Problem(obj, con)

Note we call a solver to solve `prob1` for us

In [112]:
# call solver to solve the problem
prob1.solve(verbose = True) # You may specify the solver here, 
                           # otherwise ECOS is used for LPs by default

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Oct 20 06:52:59 PM: Your problem has 3 variables, 7 constraints, and 0 parameters.
(CVXPY) Oct 20 06:52:59 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 20 06:52:59 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 20 06:52:59 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 20 06:52:59 PM: Compiling problem (target solver=OSQP).
(CVXPY) Oct 20 06:52:59 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing 

12891.666666666666

Now all solutions and status are updated into `prob1`. 

Use `prob1.status` to check the optimality

In [113]:
prob1.status

'optimal'

Optimized objective value is stored in `prob1.value`, optimized variables are stored in `x1.value`, and the Lagrange multiplier/dual value results are stored in the field `con[0].dual_value` 

**Note:**

Now we can use `print` to print out results with a nice look

In [114]:
#print optimal value and variables
print('\033[94mOptimal value: ', f'{prob1.value:.8}' +'\033[0m')
print('\033[94mOptimal variables: ', f'x1={x1.value:.5}, x2={x2.value:.5}, x3={x3.value:.5}'+'\033[0m')
print('\033[94mPrice: ', f'{con[6].dual_value:.3}'+'\033[0m')

Optimal value:  12891.667
Optimal variables:  x1=416.67, x2=83.333, x3=0.0
Price:  28.3


<font color='blue'>**Answer to Problem 2**</font>

<font color='blue'>Optimal value:  12,892

<font color='blue'>Optimal variables:  x1=417, x2=83, x3=0 so generator 1 at 417 MW, generator 2 at 83 MW, and generator 3 not running

<font color='blue'>Price:  28.3

### Problem 3 Piece-wise linear cost function (50)

A utility operates two generators with the following piece-wise linear cost curves:

**Generator 1:** The minimal generation is 50 MW and maximum is 200 MW.
* 0 MW to 100 MW: 25 \\$/MWh
* 100 MW to 150 MW: 27 \\$/MWh
* 150 MW to 200 MW: 30 \\$/MWh


**Generator 2:** The minimal generation is 30 MW and maximum is 100 MW. 
* 0 MW to 50 MW: 23 \\$/MWh
* 50 MW to 80 MW: 25 \\$/MWh
* 80 MW to 100 MW: 28 \\$/MWh


The demand to be served is 185 MW.


1) Formulate the problem into a linear proramming. Use Latex to write out your solution (refer to Problem 1 Markdown format), explain your variable definition and constraints, **do not** write code for this question. **Hint:** You should define a different variable for each segment. (20)

2) Use CVXPY to solve this problem. (20)

3) Do we need to model the segment transition logic in this case (i.e., enforce a upper segment to be zero if the lower segment is not "full" ), why? (10)

<font color='blue'>**Start of Problem 3 Part 1**</font>

<font color='blue'>**Define variables for generator 1 $p_{1}$, different variables per segment:**

$p_{11}$ for 0 MW to 100 MW segment

HOWEVER note minimum generation is 50 MW, thus

<font color='blue'>$p_{11}$ for 50 MW to 100 MW segment

<font color='blue'>$p_{12}$  for 100 MW to 150 MW segment (0 to 50 "units")

<font color='blue'>$p_{13}$  for 150 MW to 200 MW segment (0 to 50 "units")

Thus associated constaints, in UNIT form/subtract out max generation from previous segment:

<font color='blue'>$p_{11} \geq 50 $

<font color='blue'>$p_{11} \leq 100 $

<font color='blue'>$p_{12} \geq 0 $

<font color='blue'>$p_{12} \leq 50 $

<font color='blue'>$p_{13} \geq 0 $

<font color='blue'>$p_{13}  \leq 50$

With associated costs:

<font color='blue'>$c_{11} = 25 \$/MWh$

<font color='blue'>$c_{12} = 27 \$/MWh$

<font color='blue'>$c_{13} = 30 \$/MWh$

<font color='blue'>**For generator 2 $p_{2}$, different variables per segment:**

$p_{21}$ for 0 MW to 50 MW segment

HOWEVER note minimum generation is 30 MW, thus

<font color='blue'>$p_{21}$ for 30 MW to 50 MW segment

<font color='blue'>$p_{22}$  for 50 MW to 80 MW segment (0 to 30 "units")

<font color='blue'>$p_{23}$  for 80 MW to 100 MW segment (0 to 20 "units")

Thus associated constaints, in UNIT form/subtract out max generation from previous segment:

<font color='blue'>$p_{21} \geq 30 $

<font color='blue'>$p_{21} \leq 50 $

<font color='blue'>$p_{22} \geq 0 $

<font color='blue'>$p_{22} \leq 30 $

<font color='blue'>$p_{23} \geq 0 $

<font color='blue'>$p_{23}  \leq 20 $

With associated costs:

<font color='blue'>$c_{21} = 23 \$/MWh$

<font color='blue'>$c_{22} = 25 \$/MWh$

<font color='blue'>$c_{23} = 28 \$/MWh$

Total demand constraint:

<font color='blue'>$ p_{11} + p_{12} + p_{13} + p_{21} + p_{22} + p_{23} = 185 MW $

Or rearranged:

<font color='blue'>$ 185 - p_{11} - p_{12} - p_{13} - p_{21} - p_{22} - p_{23} = 0 $

<font color='blue'>**Thus objective function to be minimized is:**
$$ 25p_{11} + 27p_{12} + 30p_{13} + 23p_{21} + 25p_{22} + 28p_{23} $$

<font color='blue'>**Answer to Problem 3 Part 1: (Summarized from above)**

<font color='blue'>**Objective function to be minimized is:**
<font color='blue'>$$ 25p_{11} + 27p_{12} + 30p_{13} + 23p_{21} + 25p_{22} + 28p_{23} $$

<font color='blue'>**Subject to constraints:**

<font color='blue'>$ 185 - p_{11} - p_{12} - p_{13} - p_{21} - p_{22} - p_{23} = 0 $

<font color='blue'>$p_{11} \geq 50 $

<font color='blue'>$p_{11} \leq 100 $

<font color='blue'>$p_{12} \geq 0 $

<font color='blue'>$p_{12} \leq 50 $

<font color='blue'>$p_{13} \geq 0 $

<font color='blue'>$p_{13}  \leq 50$

<font color='blue'>$p_{21} \geq 30 $

<font color='blue'>$p_{21} \leq 50 $

<font color='blue'>$p_{22} \geq 0 $

<font color='blue'>$p_{22} \leq 30 $

<font color='blue'>$p_{23} \geq 0 $

<font color='blue'>$p_{23}  \leq 20 $

<font color='blue'>**Start of Problem 3 Part 2**</font>

In [163]:
# Define variables
p11 = cp.Variable(nonneg = True) #because won't have negative generation, not physically possible
p12 = cp.Variable(nonneg = True)
p13 = cp.Variable(nonneg = True)
p21 = cp.Variable(nonneg = True)
p22 = cp.Variable(nonneg = True)
p23 = cp.Variable(nonneg = True)

In [164]:
# Create objective function
obj3 = cp.Minimize(25*p11 + 27*p12 + 30*p13 + 23*p21 + 25*p22 + 28*p23)

In [165]:
con3 = [];

# Define constraints for generator 1
con3 += [p11 >= 50];
con3 += [p11 <= 100];
con3 += [p12 >= 0];
con3 += [p12 <= 50];
con3 += [p13 >= 0];
con3 += [p13 <= 50];
# Define constraints for generator 2
con3 += [p21 >= 30];
con3 += [p21 <= 50];
con3 += [p22 >= 0];
con3 += [p22 <= 30];
con3 += [p23 >= 0];
con3 += [p23 <= 20];
# Define demand balance constraint
con3 += [185 - p11 - p12 - p13 - p21 - p22 - p23 == 0]

In [166]:
# Solve the problem
prob3 = cp.Problem(obj3, con3)

In [167]:
# call solver to solve the problem
prob3.solve() # You may specify the solver here, 
                           # otherwise ECOS is used for LPs by default

4535.000000478926

In [168]:
prob3.status

'optimal'

In [176]:
#print optimal value and variables
print('\033[94mOptimal value: ', f'{prob3.value:.8}' +'\033[0m')
print('\033[94mOptimal variables: ', f'p11={p11.value:.5}, p12={p12.value:.5}, p13={p13.value:.5},p21={p21.value:.5},p22={p22.value:.5},p23={p23.value:.5}'+'\033[0m')
print('\033[94mPrice: ', f'{con3[-1].dual_value:.3}'+'\033[0m')

Optimal value:  4535.0
Optimal variables:  p11=100.0, p12=5.0, p13=3.8448e-08,p21=50.0,p22=30.0,p23=2.1812e-07
Price:  27.0


In [173]:
#Check: 
p11.value + p12.value + p13.value + p21.value + p22.value + p23.value

184.99999999999997

In [174]:
#Thus, generator 1 will run:
p11.value + p12.value + p13.value 

104.99999980940304

In [175]:
#Thus, generator 2 will run:
p21.value + p22.value + p23.value

80.00000019059692

<font color='blue'>**Answer to Problem 2 Part 2**</font>

<font color='blue'>**Generator 1 would total $105 MW$, generator 2 would total $80 MW$. Price would be $\$27/MWh$, which makes sense as we'd get into the second segment of generator 1 ($\$27/MWh$) and the second segment of generator 2 ($\$25/MWh$) thus generator 1 would set the price. Breakdown of values by segment printed above and summarized below.**

<font color='blue'>p11=100.0

<font color='blue'>p12=5.0

<font color='blue'>p13=0

<font color='blue'>p21=50.0

<font color='blue'>p22=30.0

<font color='blue'>p23=0

<font color='blue'>**Answer to Problem 2 Part 3**</font>

<font color='blue'>No, we don't need to model the segment transition logic in this case. Since we subtracted the generation of the previous segments from the later ones, this is already accounted for. This works because each segment has increasing cost, so the objective function will naturally minimize the later segments in favor of using the maximum possible from the earlier segments. If we hadn't defined the segment constraints in terms of "units" as opposed to cumulative generation and/or cost did not increase this way, we'd need to model the segment transition logic. 